In [8]:
# imports and some initial setup
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)

emotion_map = {
  0: 'Angry',
  1: 'Disgust',
  2: 'Fear',
  3: 'Happy',
  4: 'Sad',
  5: 'Surprise',
  6: 'Neutral'
}

train_data_raw = pd.read_csv('datasets/fer2013/train.csv')
test_data_raw = pd.read_csv('datasets/fer2013/test.csv')

# create a data generator to not load all images into memory
def data_generator(data, batch_size=32):
  while True:
    for start in range(0, len(data), batch_size):
      end = min(start + batch_size, len(data))
      batch_data = data[start:end]
      batch_features = []
      batch_labels = []
      for _, row in batch_data.iterrows():
        img = np.array(row['pixels'].split(), 'float32').reshape(48, 48)
        img /= 255.0 
        img_expanded = np.expand_dims(img, axis=-1)
        img_rgb = np.repeat(img_expanded, 3, axis=-1)
        batch_features.append(img_rgb)
        batch_labels.append(row['emotion'])
      yield np.stack(batch_features), to_categorical(batch_labels, num_classes=len(emotion_map))

train_generator = data_generator(train_data_raw)


2.17.0


In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2  # Import l2 regularizer

# Load the EfficientNetB0 model pre-trained on ImageNet, excluding the top layers
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Ensure the output shape matches the target shape
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(len(emotion_map), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
  layer.trainable = False

# adding temporary unfreeze for testing
for layer in base_model.layers[-40:]:
  layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-7), loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
  rotation_range=10,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.1,
  horizontal_flip=True,
  brightness_range=[0.8, 1.2],
  fill_mode='nearest'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

def augmented_data_generator(generator, datagen, batch_size=32):
  while True:
    # Get a batch from the custom generator
    batch_features, batch_labels = next(generator)
    # Use the datagen.flow to augment the batch
    augmented_data = datagen.flow(batch_features, batch_labels, batch_size=batch_size, shuffle=False)
    # Yield the augmented batch
    yield next(augmented_data)

train_generator_augmented = augmented_data_generator(data_generator(train_data_raw), datagen)

# Fit the model using the data generator
model.fit(
  train_generator_augmented,
  steps_per_epoch=len(train_data_raw) // 32,
  epochs=30,
  callbacks=[early_stopping, reduce_lr],
  validation_data=augmented_data_generator(data_generator(test_data_raw), datagen),
  validation_steps=len(test_data_raw) // 32,
)


Epoch 1/50
456/897 ━━━━━━━━━━━━━━━━━━━━ 1:20 182ms/step - accuracy: 0.1524 - loss: 2.7389 - precision: 0.1584 - recall: 0.0083

In [ ]:
# Evaluate the model on the test data
test_features = []
test_labels = []

for index, row in test_data_raw.iterrows():
  test_features.append(np.array(row['pixels'].split(), 'float32').reshape(48, 48, 1))
  test_labels.append(row['emotion'])

test_labels_categorical = to_categorical(test_labels, num_classes=len(emotion_map))

# Evaluate the model
evaluation = model.evaluate(np.array(test_features), test_labels_categorical)
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")

In [ ]:
from tensorflow.keras.preprocessing import image

# predict using the model on a real image
def predict_emotion(img_path):
  img = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = np.repeat(img_array, 3, axis=-1)  # Convert grayscale to 3 channels

  prediction = model.predict(img_array)
  predicted_emotion = emotion_map[np.argmax(prediction)]
  
  plt.imshow(img, cmap='gray')
  plt.title(f"Predicted Emotion: {predicted_emotion}")
  plt.axis('off')
  plt.show()

# Example usage
predict_emotion('path_to_your_image.jpg')

In [ ]:
evaluate_classification_model(model, np.array(test_features), test_labels_categorical, class_names)